# RAG Component For Feedback and Queries

This component of the project is responsible for querying any information, that is going to be extracted from the patient's electronic health record. Secondly, upon diagnosis done by the skin model, we want to take the skin disease diagnosed plus the patient's ehr to give him further reccomendations or personalized treatment plans.

### Process PDF Document

In [3]:
!pip3 install tqdm
!pip3 install PyMuPDF
!pip3 install sentence-transformers
!pip3 install -U sentence-transformers
!pip3 install tf-keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3

In [4]:
import sys
print(sys.executable)

/usr/local/Cellar/jupyterlab/4.2.1/libexec/bin/python


In [5]:
import fitz  # PyMuPDF

def simplified_pdf_preprocessor(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc[page_num]
        text += page.get_text("text") + "\n"  # Concatenate text from each page
    return text

ModuleNotFoundError: No module named 'fitz'

In [25]:
import pandas as pd

df = pd.DataFrame(pages_and_text)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,739,119,4,184.75,Patient's Electronic Health Record (EHR) 1. Pa...
1,2,828,128,9,207.00,4. Current Medications - Atorvastatin (10 mg) ...


### Chunking the text into groups of sentences

In [35]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

# def chunk_text(text, max_tokens=384, overlap=20):

#     tokens = tokenizer.tokenize(text)
#     chunks = []
#     start = 0
    
#     while start < len(tokens):
#         end = start + max_tokens
#         chunk = tokens[start:end]
        
#         chunk_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(chunk))
#         chunks.append(chunk_text)
        
#         # Slide the window forward with overlap
#         start += max_tokens - overlap

#     return chunks


# pages = [page['text'] for page in pages_and_text] 
# all_chunks = []

# for page_text in pages:
#     page_chunks = chunk_text(page_text)
#     all_chunks.extend(page_chunks)

In [36]:
# # Check the number of chunks
# print("Total number of chunks:", len(all_chunks))

# # Check the size of each chunk in terms of tokens
# for i, chunk in enumerate(all_chunks[:len(all_chunks)]):  # Show first 5 chunks as an example
#     tokens = tokenizer.tokenize(chunk)
#     print(f"Chunk {i+1} token count:", len(tokens))


Total number of chunks: 2
Chunk 1 token count: 223
Chunk 2 token count: 203


In [37]:
# from sentence_transformers import SentenceTransformer

# embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

# embeddings = embedding_model.encode(all_chunks)
# embeddings_dict = dict(zip(all_chunks, embeddings))

# # prints the two embeddings we have as a vector
# for all_chunks, embedding in embeddings_dict.items():
#     print("Sentence:", all_chunks)
#     print("Embedding:", embedding)
#     print("")

Sentence: patient ' s electronic health record ( ehr ) 1. patient information full name : john doe medical id : 123 - 45 - 6789 date of birth : march 15, 1980 ( age : 44 ) address : 123 elm street, springfield, il phone number : ( 555 ) 123 - 4567 2. family medical history mother : hypertension, type 2 diabetes father : coronary artery disease, high cholesterol siblings : no significant medical history 3. blood reports and lab tests cbc ( complete blood count ) - march 5, 2024 - white blood cell count ( wbc ) : 5. 7 k / ul - hemoglobin : 14. 2 g / dl - platelets : 250 k / ul lipid profile - march 10, 2024 - total cholesterol : 190 mg / dl - ldl : 120 mg / dl - hdl : 45 mg / dl - triglycerides : 160 mg / dl blood glucose tests - fasting glucose : 95 mg / dl - hba1c ( glycated hemoglobin ) : 5. 8 %
Embedding: [ 2.91272402e-02  2.41231732e-02  1.33620538e-02 -3.55138332e-02
 -8.95194337e-03  3.75075340e-02 -2.74995156e-02  3.58690284e-02
 -3.87865421e-03  4.01588269e-02  4.13790010e-02 -2

In [43]:
# embedding_model.to("cpu")

# embeddings = embedding_model.encode(all_chunks)
# embeddings_dict = dict(zip(all_chunks, embeddings))

# # Convert the dictionary to a DataFrame
# text_chunks_and_embeddings_df = pd.DataFrame({
#     'text_chunk': list(embeddings_dict.keys()),
#     'embedding': [embedding.tolist() for embedding in embeddings_dict.values()]
# })

# # Save the DataFrame to a CSV
# embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
# text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

# # Load the DataFrame to verify
# text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
# print(text_chunks_and_embedding_df_load.head())

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


OSError: [Errno 28] No space left on device: 'text_chunks_and_embeddings_df.csv'

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')
chunk_embeddings = embedding_model.encode(all_chunks, convert_to_tensor=True)

def answer_query(query, model, chunk_embeddings, chunks):
    """Finds the most relevant chunk and answers the query."""
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(query_embedding, chunk_embeddings)[0]
    best_match_idx = similarity_scores.argmax().item()
    
    # Return the most relevant chunk as the answer
    answer = chunks[best_match_idx]
    return answer

# Test the setup with a sample query
query = "What medications is the patient currently taking?"
answer = answer_query(query, embedding_model, chunk_embeddings, all_chunks)
print("Answer:", answer)
